In [5]:
import pandas as pd
df = pd.read_csv('ImpexiumInsights.csv')
df.head(10)

# Change client IDs to a more readible format as the following:
# Client_1 -> ICSC
# Client_2 -> SIOR
# Client_3 -> NAIOP

df['Client'] = df['Client'].replace('Client_1', 'ICSC')
df['Client'] = df['Client'].replace('Client_2', 'SIOR')
df['Client'] = df['Client'].replace('Client_3', 'NAIOP')
df.head(10)

# Saving updated data frame to the csv

df.to_csv('modified_impexiumInsights.csv', index=False)